This Note Book is same as app.py file
I just used Note Book features to implement its functions :)

In [1]:
from ultralytics import YOLO
import pandas as pd
import numpy as np
from PIL import Image
import io
# from ultralytics.yolo.utils.plotting import Annotator, colors
from ultralytics.utils.plotting import Annotator, colors

In [2]:
model = YOLO('models/yolov8n.pt')

In [6]:
result = model.predict(source='bus.jpg', conf=0.6)


image 1/1 e:\projects\Python\YOLO-FastAPI\bus.jpg: 640x480 3 persons, 1 bus, 184.0ms
Speed: 23.0ms preprocess, 184.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)


In [23]:
result[0].names

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

In [11]:
result[0].to('cpu').numpy().boxes.xyxy

array([[     22.871,      231.28,         805,      756.84],
       [      48.55,      398.55,      245.35,       902.7],
       [     669.47,      392.19,      809.72,      877.04],
       [     221.52,       405.8,      344.97,      857.54]], dtype=float32)

In [14]:
bboxes = pd.DataFrame(result[0].to('cpu').numpy().boxes.xyxy, columns=['x1', 'y1', 'x2', 'y2'])

In [17]:
bboxes['confs'] = result[0].to('cpu').numpy().boxes.conf

In [20]:
bboxes['classes'] = (result[0].to('cpu').numpy().boxes.cls).astype(int)

In [26]:
bboxes['classes'] = bboxes['classes'].replace(result[0].names)

In [27]:
bboxes

,x1,y1,x2,y2,confs,classes
0,22.871269,231.277313,805.002686,756.840454,0.873449,bus
1,48.550465,398.552216,245.345596,902.702698,0.865691,person
2,669.472900,392.185974,809.720032,877.035461,0.852835,person
3,221.517288,405.798645,344.970612,857.536621,0.825224,person


In [2]:
# Key function 1

model = YOLO('models/yolov8n.pt')
def get_predict_df(file : Image) -> pd.DataFrame:
    # model = YOLO('models/yolov8n.pt')

    result = model.predict(source=file, conf=0.6)
    bboxes = pd.DataFrame(result[0].to('cpu').numpy().boxes.xyxy, columns=['x1', 'y1', 'x2', 'y2'])
    bboxes['conf'] = result[0].to('cpu').numpy().boxes.conf
    bboxes['class'] = (result[0].to('cpu').numpy().boxes.cls).astype(int)
    bboxes['name'] = bboxes['class'].replace(result[0].names)
    
    return bboxes

In [4]:
img = Image.open('bus.jpg').convert("RGB")

In [5]:
preds = get_predict_df(img)


0: 640x480 3 persons, 1 bus, 209.0ms
Speed: 8.0ms preprocess, 209.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)


In [6]:
out_img = draw_boxes(img=img, preds=preds)

In [7]:
out_img.show()

In [5]:
annotator = Annotator(np.array(img))

In [6]:
preds = preds.sort_values(by=['x1'], ascending=True)

In [7]:
for i, data in preds.iterrows():
    # print(f"index= {i}")
    # print(f"data:\n{data}")
    # print('##########\n')
    annotator.box_label(
        box=[
            data['x1'],data['y1'],
            data['x2'],data['y2']
        ],
        label=f"{data['name']}: {data['conf']:.2f}", 
        color=colors(data['class'], bgr=True))

In [8]:
final_img2 = Image.fromarray(annotator.result())

In [9]:
final_img2.show()

In [3]:
# Key function 2

def draw_boxes(img : Image, preds:pd.DataFrame) -> Image:
    annotator = Annotator(np.array(img))
    preds = preds.sort_values(by=['x1'], ascending=True)
    for i, data in preds.iterrows():
        annotator.box_label(
            box=[
                data['x1'],data['y1'],
                data['x2'],data['y2']
            ],
            label=f"{data['name']}: {data['conf']:.2f}", 
            color=colors(data['class'], bgr=True))
    return Image.fromarray(annotator.result())

In [3]:
seg_model = YOLO('models/yolov8n-seg.pt')

In [13]:
# key function 3
def get_segmented_img(file:Image) -> Image:
    result = seg_model.predict(source=file, conf=0.6)[0]
    out_img = Image.fromarray(result.plot()[..., ::-1])
    return out_img

In [14]:
img = Image.open('pics/bus.jpg').convert("RGB")

In [15]:
out_img = get_segmented_img(file=img)


0: 640x480 3 persons, 1 bus, 236.1ms
Speed: 6.0ms preprocess, 236.1ms inference, 11.0ms postprocess per image at shape (1, 3, 640, 480)


In [16]:
out_img.show()

In [6]:
result = seg_model.predict(source=img, conf=0.6)


0: 640x480 3 persons, 1 bus, 978.7ms
Speed: 332.7ms preprocess, 978.7ms inference, 311.0ms postprocess per image at shape (1, 3, 640, 480)


In [11]:
r = result[0]
im_array = r.plot()  # plot a BGR numpy array of predictions
im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
im.show()  # show image
# im.save('results.jpg')  # save image

In [2]:
class_model = YOLO('models/yolov8n-cls.pt')

In [3]:
img = Image.open('pics/bus.jpg').convert("RGB")

In [4]:
result = class_model.predict(source=img, conf=0.6)


0: 224x224 minibus 0.47, police_van 0.25, ambulance 0.05, recreational_vehicle 0.04, bullet_train 0.02, 600.7ms
Speed: 389.6ms preprocess, 600.7ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)


In [15]:
result[0].probs

ultralytics.engine.results.Probs object with attributes:

data: tensor([2.2782e-07, 3.9835e-07, 1.8025e-05, 1.2990e-06, 4.0058e-05, 4.2587e-06, 1.5845e-06, 5.4020e-06, 1.4168e-07, 1.0631e-06, 1.3268e-08, 5.1390e-08, 5.9792e-08, 4.9972e-09, 3.2175e-07, 3.0764e-08, 4.8649e-09, 1.1995e-06, 8.7467e-07, 5.7113e-08, 2.0526e-09, 7.0013e-08, 1.6223e-07, 1.2153e-07, 3.2795e-08, 1.7966e-07,
        2.7880e-08, 2.7480e-08, 6.9575e-08, 2.4716e-06, 1.9853e-07, 7.1315e-09, 9.2888e-08, 1.1460e-05, 2.4519e-05, 2.9449e-07, 9.7188e-07, 2.1076e-07, 1.8162e-08, 2.7766e-06, 8.3557e-08, 1.0282e-07, 1.3435e-07, 9.6201e-08, 1.0423e-07, 2.0541e-06, 1.0941e-07, 1.2530e-07, 3.9443e-06, 6.0217e-07, 4.4510e-06, 2.0545e-05,
        1.6594e-07, 7.0468e-08, 2.3901e-07, 1.1954e-07, 3.0200e-07, 2.0979e-08, 1.1534e-07, 1.8255e-07, 3.8974e-06, 7.5831e-06, 4.5554e-06, 1.1325e-06, 8.9009e-07, 2.7293e-06, 4.6082e-08, 9.5021e-06, 5.4790e-08, 2.2610e-08, 1.3181e-09, 3.7680e-07, 2.9976e-08, 6.0336e-08, 6.4248e-08, 9.7971e-09, 

In [12]:
result[0].to('cpu').numpy().probs.top5

[654, 734, 407, 757, 466]

In [13]:
classes = pd.DataFrame(result[0].to('cpu').numpy().probs.top5, columns=['class'])

In [18]:
classes['conf'] = result[0].to('cpu').numpy().probs.top5conf

In [20]:
classes['name'] = classes['class'].replace(result[0].names)

In [21]:
classes

,class,conf,name
0,654,0.466905,minibus
1,734,0.246027,police_van
2,407,0.049342,ambulance
3,757,0.042209,recreational_vehicle
4,466,0.024124,bullet_train


In [2]:
class_model = YOLO('models/yolov8n-cls.pt')

def get_class_df(file:Image) -> pd.DataFrame:
    result = class_model.predict(source=file, conf=0.6)
    classes = pd.DataFrame(result[0].to('cpu').numpy().probs.top5, columns=['class'])
    classes['conf'] = result[0].to('cpu').numpy().probs.top5conf
    classes['name'] = classes['class'].replace(result[0].names)
    return classes

In [3]:
img = Image.open('pics/bus.jpg').convert("RGB")

In [4]:
classes = get_class_df(file=img)
classes


0: 224x224 minibus 0.47, police_van 0.25, ambulance 0.05, recreational_vehicle 0.04, bullet_train 0.02, 28.0ms
Speed: 3.0ms preprocess, 28.0ms inference, 1.0ms postprocess per image at shape (1, 3, 224, 224)


,class,conf,name
0,654,0.466905,minibus
1,734,0.246027,police_van
2,407,0.049342,ambulance
3,757,0.042209,recreational_vehicle
4,466,0.024124,bullet_train


In [2]:
pose_model = YOLO('models/yolov8n-pose.pt')

In [3]:
img = Image.open('pics/bus.jpg').convert("RGB")


In [4]:
result = pose_model.predict(source=img)


0: 640x480 4 persons, 839.8ms
Speed: 158.8ms preprocess, 839.8ms inference, 362.0ms postprocess per image at shape (1, 3, 640, 480)


In [13]:
annotator = Annotator(np.array(img))

In [49]:
for k in reversed(result[0].keypoints.data):
            annotator.kpts(k, result[0].orig_shape, radius=5, kpt_line=True)

In [ ]:
# annotator.kpts(kpts, shape=result[0].orig_shape, radius=5, kpt_line=True)

In [50]:
out = Image.fromarray(annotator.result())
out.show()

In [2]:
# key function
pose_model = YOLO('models/yolov8n-pose.pt')

def get_posed_img(file:Image) ->Image:
    result = pose_model.predict(source=file)
    annotator = Annotator(np.array(file))
    for k in reversed(result[0].keypoints.data):
        annotator.kpts(k, result[0].orig_shape, radius=5, kpt_line=True)
    return Image.fromarray(annotator.result())

In [3]:
img = Image.open('pics/bus.jpg').convert("RGB")

In [4]:
out = get_posed_img(file=img)
out.show()


0: 640x480 4 persons, 199.0ms
Speed: 5.2ms preprocess, 199.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)
